# **```Supervised Neural Machine Translation```**

This notebook runs Seq2Seq model to translate Sanskrit to English.

NOTE: This requires GPU to run. With only CPU, processing can be painfully slow, with GPU also, training takes around 1.5 hrs

In [ ]:
!rm -rf sample_data
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This is the repo which contains the parallel data we are going to use.

In [ ]:
pri

In [ ]:
%cd /content/drive/MyDrive
!git clone https://github.com/priyanshu2103/Sanskrit-Hindi-Machine-Translation.git

/content/drive/MyDrive
Cloning into 'Sanskrit-Hindi-Machine-Translation'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 124 (delta 46), reused 58 (delta 12), pack-reused 0
Receiving objects: 100% (124/124), 9.25 MiB | 11.28 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [ ]:
%cd /content/drive/MyDrive/Sanskrit-Hindi-Machine-Translation/parallel-corpus/sanskrit-english/

eng_lines = []
sanskrit_lines = []
with open('bhagvadgita_english.txt', 'r') as f:
  eng_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open('bible_english.txt', 'r') as f:
  eng_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open('manu_english.txt', 'r') as f:
  eng_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open('ramayan_english.txt', 'r') as f:
  eng_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open('rigveda_english.txt', 'r') as f:
  eng_lines.extend([x.replace('\n', '') for x in f.readlines()])



with open('bhagvadgita_sanskrit.txt', 'r') as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open('bible_sanskrit.txt', 'r') as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open('manu_sanskrit.txt', 'r') as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open('ramayan_sanskrit.txt', 'r') as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

with open('rigveda_sanskrit.txt', 'r') as f:
  sanskrit_lines.extend([x.replace('\n', '') for x in f.readlines()])

# print(eng_lines[:100])
# print(sanskrit_lines[:100])

print(len(eng_lines))
print(len(sanskrit_lines))

/content/drive/MyDrive/Sanskrit-Hindi-Machine-Translation/parallel-corpus/sanskrit-english
34374
34374


In [ ]:
# Randomly shuffling the data into training and dev set, the test data is already provided on github
import random
c = list(zip(sanskrit_lines, eng_lines))
random.shuffle(c)

sanskrit_lines, eng_lines = zip(*c)

train_text_sa = sanskrit_lines[:-1374]
train_text_en = eng_lines[:-1374]

dev_text_sa = sanskrit_lines[-1374:]
dev_text_en = eng_lines[-1374:]

In [ ]:
print(len(train_text_en))
print(len(dev_text_en))

33000
1374


In [ ]:
# Start and End tokens
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Reading the data and forming pairs
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Forming pairs of sentences
    pairs = [[train_text_sa[i], train_text_en[i]] for i in range(len(train_text_sa))]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Filtering the data by only keeping sentences of length less than 100 words




In [ ]:
MAX_LENGTH = 100

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('sanskrit', 'eng', False)
print(random.choice(pairs)) # a random pair of parallel sentences

Reading lines...
Read 33000 sentence pairs
Trimmed to 32936 sentence pairs
Counting words...
Counted words:
sanskrit 124262
eng 44867
['आयतीमिव विध्वस्तामाज्ञां प्रतिहतामिव। दीप्तामिव दिशं काले पूजामपहृतामिव।।', 'She was like a prospect blasted, command flouted, entire direction set on flames at the time of catastrophe and articles of worship abducted by evil forces at the sacrificial ceremony.']


Seq2Seq Model
=================

A seq2seq network is a model consisting of two RNNs called the encoder and decoder. The encoder reads an input sequence and outputs a single vector, and the decoder reads that vector to produce an output sequence.

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.


The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




Attention Decoder

If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.

In [ ]:
import random
def indexesFromSentence(lang, sentence):
    res = []
    for word in sentence.split(' '):
      if word not in lang.word2index.keys():
        res.append(random.choice(list(lang.word2index.values())))
      else:
        res.append(lang.word2index[word])
    return res

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.




In [ ]:
teacher_forcing_ratio = 0.5
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]

    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
# Print time elapsed and estimated time remaining given the current time and progress %.
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
# Each iteration of training
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    # showPlot(plot_losses)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there.

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

Google translate API to translate English to Hindi

In [ ]:
$ pip install googletrans


In [ ]:
!pip install googletrans
from googletrans import Translator
translator = Translator()

In [ ]:
!pip install googletrans


In [ ]:
dev_text_sa = sanskrit_lines[-1374:]
dev_text_en = eng_lines[-1374:]

In [ ]:
def evaluate_test(encoder, decoder):
  test = []
  # with open('/content/drive/MyDrive/Sanskrit-Hindi-Machine-Translation/parallel-corpus/sanskrit-english/bhagvadgita_sanskrit.txt', 'r') as f:
  #test_text = [x.replace('\n', '') for x in dev_text_sa]

  for sa_text in dev_text_sa:
    # print('>', pair[0])
    # print('=', pair[1])
    output_words, attentions = evaluate(encoder, decoder, sa_text)
    output_sentence = ' '.join(output_words)
    #print('<', output_sentence)
    # print('')
    test.append(output_sentence)
    #print(test)

  # translated = []
  # print(len(test))

  # #for line in test:
  # ans = translator.translate(test, dest='hi',src='auto')
  # print(ans)

  #translated.append(ans.text)

  with open('/content/drive/MyDrive/Sanskrit-Hindi-Machine-Translation/parallel-corpus/sanskrit-hindi/translated_eng.en', 'w') as f:
    for line in test:
      f.write(line + '\n')

  # calculate bleu score
  import nltk
  #with open('/content/drive/MyDrive/Sanskrit-Hindi-Machine-Translation/parallel-corpus/sanskrit-english/bhagvadgita_english.txt', 'r') as f: # ground truth file
  #reference = [x.replace('\n', '') for x in dev_text_en]
  reference=dev_text_en
  with open('/content/drive/MyDrive/Sanskrit-Hindi-Machine-Translation/parallel-corpus/sanskrit-hindi/translated_eng.en', 'r') as f: # translated hindi file (our output)
    candidate = [x.replace('\n', '') for x in f.readlines()]

  from nltk.translate.bleu_score import SmoothingFunction
  smoothie = SmoothingFunction().method4

  #print("bleu", nltk.translate.bleu_score.corpus_bleu(reference, candidate, smoothing_function=smoothie))
  print("bleu", nltk.translate.bleu_score.corpus_bleu(reference, candidate, smoothing_function=smoothie))
  print("bleu1", nltk.translate.bleu_score.corpus_bleu(reference, candidate,weights=(1,0,0,0), smoothing_function=smoothie))
  print("bleu2", nltk.translate.bleu_score.corpus_bleu(reference, candidate,weights=(0,1,0,0), smoothing_function=smoothie))
  print("bleu3", nltk.translate.bleu_score.corpus_bleu(reference, candidate,weights=(0,0,1,0), smoothing_function=smoothie))
  print("bleu4", nltk.translate.bleu_score.corpus_bleu(reference, candidate,weights=(1,0,0,1), smoothing_function=smoothie))
  print("chrf",nltk.translate.chrf_score.corpus_chrf(reference, candidate))

In [ ]:
# defining hidden layer size and the encoder and decoder networks
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

# train
trainIters(encoder1, attn_decoder1, 80000, print_every=5000)

6m 43s (- 100m 58s) (5000 6%) 6.2418
13m 12s (- 92m 25s) (10000 12%) 5.9179
19m 45s (- 85m 35s) (15000 18%) 5.8198
26m 26s (- 79m 18s) (20000 25%) 5.8037
33m 12s (- 73m 2s) (25000 31%) 5.6742
39m 59s (- 66m 39s) (30000 37%) 5.6593
46m 39s (- 59m 58s) (35000 43%) 5.5642
53m 17s (- 53m 17s) (40000 50%) 5.4875
60m 5s (- 46m 44s) (45000 56%) 5.4830
66m 50s (- 40m 6s) (50000 62%) 5.4655
73m 34s (- 33m 26s) (55000 68%) 5.4487
80m 12s (- 26m 44s) (60000 75%) 5.3887
86m 59s (- 20m 4s) (65000 81%) 5.4371
93m 41s (- 13m 23s) (70000 87%) 5.3510
100m 27s (- 6m 41s) (75000 93%) 5.3624
107m 10s (- 0m 0s) (80000 100%) 5.3293


In [ ]:
# saving the trained models
torch.save(encoder1.state_dict(), '/content/drive/MyDrive/Sanskrit-Hindi-Machine-Translation/trained_encoder.pth')
torch.save(attn_decoder1.state_dict(), '/content/drive/MyDrive/Sanskrit-Hindi-Machine-Translation/trained_decoder.pth')

In [ ]:
evaluate_test(encoder1, attn_decoder1)

bleu 7.834621847813588e-05
bleu1 0.15241216403141028
bleu2 1.2364869723516339e-05
bleu3 6.275543412299154e-06
bleu4 4.855469987104252e-07
chrf 0.10105958901085928


In [ ]:
#with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/test/src-test.txt-filtered.sa', 'r') as f:

test_text = dev_text_sa
#with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/test/tgt-test.txt-filtered.hi', 'r') as f: # ground truth file
reference = dev_text_en

with open('/content/drive/MyDrive/Sanskrit-Hindi-Machine-Translation/parallel-corpus/sanskrit-hindi/translated_eng.en', 'r') as f: # translated hindi file (our output)
    candidate = [x.replace('\n', '') for x in f.readlines()]
for i in range(len(test_text)):
  print("source sanskrit:",test_text[i])
  print("actual english:",reference[i])
  print("predicted english:",candidate[i])

source sanskrit: आदितो युष्माभि र्यत् श्रुतं तद् युष्मासु तिष्ठतु, आदितः श्रुतं वाक्यं यदि युष्मासु तिष्ठति, तर्हि यूयमपि पुत्रे पितरि च स्थास्यथ।
actual english: Let that therefore abide in you, which ye have heard from the beginning. If that which ye have heard from the beginning shall remain in you, ye also shall continue in the Son, and in the Father.
predicted english: Let him not and and and the and and and the and and the and the and <EOS>
source sanskrit: अकारणे परित्यक्ता मातापित्रोर्गुरोस्तथा । अकारणपरित्यक्ताब्राह्मैर्यौनैश्च सम्बन्धैः संयोगं पतितैर्गतः
actual english: He who forsakes his mother, his father, or a teacher without a (sufficient) reason, he who has contracted an alliance with outcasts either through the Veda or through a marriage, 
predicted english: The king of of of and and the and the the and the the and the and the the and the the <EOS>
source sanskrit: ता नो हिन्वन्त्वध्वरम|
actual english: speed forth this sacrifice of ours.
predicted english: and the our

In [ ]:
### for spearman correlation between hum eval and BLEU1,2,3,4 and chrf####

In [ ]:
import nltk
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4
b1=[]
b2=[]
b3=[]
b4=[]
chRf=[]
#with open('/content/drive/MyDrive/sa_hi_zst1_25gb/data/test/src-test.txt-filtered.sa', 'r') as f:
reference = dev_text_en

with open('/content/drive/MyDrive/Sanskrit-Hindi-Machine-Translation/parallel-corpus/sanskrit-hindi/translated_eng.en', 'r') as f: # translated hindi file (our output)
    candidate = [x.replace('\n', '') for x in f.readlines()]
for i in range(0,50):
  b1.append(nltk.translate.bleu_score.sentence_bleu(reference[i], candidate[i],weights=(1,0,0,0), smoothing_function=smoothie))
  b2.append(nltk.translate.bleu_score.sentence_bleu(reference[i], candidate[i],weights=(0,1,0,0), smoothing_function=smoothie))
  b3.append(nltk.translate.bleu_score.sentence_bleu(reference[i], candidate[i],weights=(0,0,1,0), smoothing_function=smoothie))
  b4.append(nltk.translate.bleu_score.sentence_bleu(reference[i], candidate[i],weights=(0,0,0,1), smoothing_function=smoothie))
  chRf.append(nltk.translate.chrf_score.sentence_chrf(reference[i], candidate[i]))

In [ ]:
print(b1)
print(b2)
print(b3)
print(b4)
print(chRf)

[0.1643835616438356, 0.14285714285714285, 0.4, 0.2391304347826087, 0.07547169811320754, 0.18461538461538465, 0.18644067796610173, 0.11111111111111109, 0.1320754716981132, 0.16417910447761194, 0.13043478260869565, 0.1724137931034483, 0.2222222222222222, 0.3, 0.27586206896551724, 0.22033898305084745, 0.140625, 0.15873015873015872, 0.16455696202531644, 0.20312499999999997, 0.1046511627906977, 0.16666666666666669, 0.276595744680851, 0.21739130434782608, 0.14285714285714285, 0.13559322033898305, 0.15555555555555556, 0.20689655172413796, 0.23076923076923078, 0.15151515151515155, 0.12280701754385966, 0.23255813953488372, 0.10526315789473682, 0.3409090909090909, 0.06557377049180328, 0.23404255319148937, 0.12280701754385966, 0.1311475409836066, 0.1126760563380282, 0.08163265306122448, 0.09677419354838708, 0.18666666666666668, 0.35294117647058826, 0.12500000000000003, 0.19642857142857142, 0.12280701754385966, 0.23529411764705885, 0.2037037037037037, 0.16923076923076924, 0.13333333333333333]
[0.0

In [ ]:
def rankify(X):

	N = len(X)

	# Rank Vector
	Rank_X = [None for _ in range(N)]

	for i in range(N):

		r = 1
		s = 1

		# Count no of smaller elements
		# in 0 to i-1
		for j in range(i):
			if (X[j] < X[i]):
				r += 1
			if (X[j] == X[i]):
				s += 1

		# Count no of smaller elements
		# in i+1 to N-1
		for j in range(i+1, N):
			if (X[j] < X[i]):
				r += 1
			if (X[j] == X[i]):
				s += 1

		# Use Fractional Rank formula
		# fractional_rank = r + (n-1)/2
		Rank_X[i] = r + (s-1) * 0.5

	# Return Rank Vector
	return Rank_X

In [ ]:
rank_y=rankify(chRf)

In [ ]:
#printVector(rank_y)
hum4=[]
hum3=[]
hum2=[]
hum1=[]
#print(rank_y)(rank_y) is of chrfscore
for i in range(len(rank_y)):
   if rank_y[i]>=45:
    hum4.append(i)
   elif rank_y[i]>=40 and  rank_y[i]<45:
    hum3.append(i)
   elif rank_y[i]>=25 and rank_y[i]<40:
    hum2.append(i)
   else:
    hum1.append(i)

print(hum1)
print(hum2)
print(hum3)
print(hum4)


[2, 3, 4, 5, 6, 7, 8, 11, 14, 18, 20, 21, 22, 23, 26, 33, 34, 35, 36, 39, 40, 42, 43, 46]
[0, 10, 12, 13, 17, 25, 27, 28, 30, 31, 37, 38, 45, 47, 48]
[1, 9, 15, 16, 32]
[19, 24, 29, 41, 44, 49]


In [ ]:
### humrank calculated for 50 sentences from value 1 to 4
humval=[]
for i in hum1:
    humval.insert(i,1)
for i in hum2:
    humval.insert(i,2)
for i in hum3:
    humval.insert(i,3)
for i in hum4:
    humval.insert(i,4)
print(humval)


[2, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 2, 3, 3, 2, 1, 4, 1, 1, 2, 1, 4, 1, 1, 1, 1, 4, 1, 1, 2, 1, 2, 3, 2, 1, 2, 2, 1, 4, 1, 2, 4, 2, 2, 2, 2, 4]


In [ ]:
humval=[2, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 2, 3, 3, 2, 1, 4, 1, 1, 2, 1, 4, 1, 1, 1, 1, 4, 1, 1, 2, 1, 2, 3, 2, 1, 2, 2, 1, 4, 1, 2, 4, 2, 2, 2, 2, 4]


In [ ]:
b1=[0.1643835616438356, 0.14285714285714285, 0.4, 0.2391304347826087, 0.07547169811320754, 0.18461538461538465, 0.18644067796610173, 0.11111111111111109, 0.1320754716981132, 0.16417910447761194, 0.13043478260869565, 0.1724137931034483, 0.2222222222222222, 0.3, 0.27586206896551724, 0.22033898305084745, 0.140625, 0.15873015873015872, 0.16455696202531644, 0.20312499999999997, 0.1046511627906977, 0.16666666666666669, 0.276595744680851, 0.21739130434782608, 0.14285714285714285, 0.13559322033898305, 0.15555555555555556, 0.20689655172413796, 0.23076923076923078, 0.15151515151515155, 0.12280701754385966, 0.23255813953488372, 0.10526315789473682, 0.3409090909090909, 0.06557377049180328, 0.23404255319148937, 0.12280701754385966, 0.1311475409836066, 0.1126760563380282, 0.08163265306122448, 0.09677419354838708, 0.18666666666666668, 0.35294117647058826, 0.12500000000000003, 0.19642857142857142, 0.12280701754385966, 0.23529411764705885, 0.2037037037037037, 0.16923076923076924, 0.13333333333333333]
b2=[0.005958971446039433, 0.005012066118352053, 0.0134119826036175, 0.008508091992197986, 0.007635176756831004, 0.006522480109211932, 0.0070302369722512435, 0.008651505658568908, 0.007635176756831004, 0.006370746393016614, 0.008508091992197986, 0.007123584229028806, 0.008651505658568908, 0.009458665266958812, 0.012026056535665977, 0.0070302369722512435, 0.006601401719618529, 0.006682475365147634, 0.005601856221111566, 0.006601401719618529, 0.005240408583827657, 0.006445622680040651, 0.008369886090674036, 0.008508091992197986, 0.008107958954397138, 0.0070302369722512435, 0.008651505658568908, 0.007123584229028806, 0.006522480109211932, 0.006445622680040651, 0.007219734406847412, 0.00895523837069896, 0.004844549884681426, 0.008800441009112238, 0.006851456440288851, 0.008369886090674036, 0.007219734406847412, 0.006851456440288851, 0.006089542681487594, 0.008107958954397138, 0.006765794073844415, 0.005834443396670688, 0.017707583400351348, 0.008236597895548708, 0.007318821255882091, 0.007219734406847412, 0.01068594098368534, 0.007526384993517495, 0.006522480109211932, 0.004474961875151464]
b3=[0.003021450310667881, 0.002534190733998231, 0.00699755614101783, 0.004350728859646699, 0.003892443052502081, 0.0033130057697584405, 0.003576787231496245, 0.004426351732291069, 0.003892443052502081, 0.0032343789379930495, 0.004350728859646699, 0.00362539554513073, 0.004426351732291069, 0.004853788755413074, 0.006235733018493471, 0.003576787231496245, 0.003353937970451348, 0.0033960120708127314, 0.002837303800303262, 0.003353937970451348, 0.0026513972001508965, 0.003273167767208146, 0.004277941779677842, 0.004350728859646699, 0.004140234359692155, 0.003576787231496245, 0.004426351732291069, 0.00362539554513073, 0.0033130057697584405, 0.003273167767208146, 0.0036755011525768646, 0.004586829409382392, 0.0024483209094626575, 0.0045049876594265015, 0.003483791410316366, 0.004277941779677842, 0.0036755011525768646, 0.003483791410316366, 0.0030888984616241413, 0.004140234359692155, 0.003439278654204242, 0.0029571836394084324, 0.009444044480187383, 0.0042078271857694475, 0.0037271774914214356, 0.0036755011525768646, 0.005509938319712753, 0.0038355615832348804, 0.0033130057697584405, 0.0022592040534745246]
b4=[0.001532306943267282, 0.001281494177987741, 0.0036578134373502283, 0.0022259543002843573, 0.001985145956776061, 0.001683220673345015, 0.00182032921602934, 0.0022658705296251893, 0.001985145956776061, 0.0016424580544495955, 0.0022259543002843573, 0.0018456559138847355, 0.0022658705296251893, 0.0024924861176445524, 0.0032377844519100724, 0.00182032921602934, 0.001704460280065439, 0.0017263061359964725, 0.0014373183725220461, 0.001704460280065439, 0.0013416708723655141, 0.0016625614055660418, 0.002187583864607988, 0.0022259543002843573, 0.002115119727234035, 0.00182032921602934, 0.0022658705296251893, 0.0018456559138847355, 0.001683220673345015, 0.0016625614055660418, 0.0018717829943678478, 0.0023507500723084773, 0.0012374665466305824, 0.002307432703608697, 0.001771928389729875, 0.002187583864607988, 0.0018717829943678478, 0.001771928389729875, 0.0015671617195004836, 0.002115119727234035, 0.0017487857563750389, 0.0014991278172001078, 0.005059309542957528, 0.002150667228282162, 0.0018987507975165793, 0.0018717829943678478, 0.00284383913275497, 0.001955384336551114, 0.001683220673345015, 0.0011406765564111576]
chRf=[0.09824218377600359, 0.11974866411343983, 0.03933931000081217, 0.055876214814486215, 0.08078216208530709, 0.07571371550298614, 0.07533911596503647, 0.04854520576828788, 0.0638758309359518, 0.10841861393944745, 0.0930678350892926, 0.04459533580847811, 0.10461504939235766, 0.10680421812296732, 0.025437674508483585, 0.11369941656062253, 0.10879017071719754, 0.10271950235186471, 0.07353012738186944, 0.24964818596121255, 0.08686237157704992, 0.08064948871114716, 0.07956775401486797, 0.06512650226355914, 0.15407177841524658, 0.10256914281771495, 0.0654367878459035, 0.09791611737448368, 0.10211359202134003, 0.16386148380678714, 0.08838978487775079, 0.08837563499287336, 0.10917812142038953, 0.0843755672516079, 0.05332027571835544, 0.05696138764784902, 0.07856587652243034, 0.09992462818060045, 0.10250721788635098, 0.042037399214156196, 0.048843113460862266, 0.1509380668851631, 0.05531647964685745, 0.042186097462083766, 0.13093746597464073, 0.09415510713892873, 0.04017327246643572, 0.09770462368261229, 0.09861227671780332, 0.14153499096117636]

In [ ]:
# Python3 Program to find correlation coefficient


# Utility Function to print
# a Vector
def printVector(X):
	print(*X)

# Function returns the rank vector
# of the set of observations


def rankify(X):

	N = len(X)

	# Rank Vector
	Rank_X = [None for _ in range(N)]

	for i in range(N):

		r = 1
		s = 1

		# Count no of smaller elements
		# in 0 to i-1
		for j in range(i):
			if (X[j] < X[i]):
				r += 1
			if (X[j] == X[i]):
				s += 1

		# Count no of smaller elements
		# in i+1 to N-1
		for j in range(i+1, N):
			if (X[j] < X[i]):
				r += 1
			if (X[j] == X[i]):
				s += 1

		# Use Fractional Rank formula
		# fractional_rank = r + (n-1)/2
		Rank_X[i] = r + (s-1) * 0.5

	# Return Rank Vector
	return Rank_X


# function that returns
# Pearson correlation coefficient.
def correlationCoefficient(X, Y):
	n = len(X)
	sum_X = 0
	sum_Y = 0
	sum_XY = 0
	squareSum_X = 0
	squareSum_Y = 0

	for i in range(n):

		# sum of elements of array X.
		sum_X = sum_X + X[i]

		# sum of elements of array Y.
		sum_Y = sum_Y + Y[i]

		# sum of X[i] * Y[i].
		sum_XY = sum_XY + X[i] * Y[i]

		# sum of square of array elements.
		squareSum_X = squareSum_X + X[i] * X[i]
		squareSum_Y = squareSum_Y + Y[i] * Y[i]

	# use formula for calculating
	# correlation coefficient.
	corr = (n * sum_XY - sum_X * sum_Y) / ((n * squareSum_X -
											sum_X * sum_X) * (n * squareSum_Y - sum_Y * sum_Y)) ** 0.5

	return corr


# Driver function
X = b1
Y =humval

# Get ranks of vector X
rank_x = rankify(X)

# Get ranks of vector y
rank_y = rankify(Y)

print("Vector X")
printVector(X)

# Print rank vector of X
print("Rankings of X")
printVector(rank_x)

# Print Vector Y
print("Vector Y")
printVector(Y)

# Print rank vector of Y
print("Rankings of Y")
printVector(rank_y)

# Print Spearmans coefficient
print("Spearman's Rank correlation: ")
print(correlationCoefficient(rank_x, rank_y))


# This code is contributed by phasing17


Vector X
0.1643835616438356 0.14285714285714285 0.4 0.2391304347826087 0.07547169811320754 0.18461538461538465 0.18644067796610173 0.11111111111111109 0.1320754716981132 0.16417910447761194 0.13043478260869565 0.1724137931034483 0.2222222222222222 0.3 0.27586206896551724 0.22033898305084745 0.140625 0.15873015873015872 0.16455696202531644 0.20312499999999997 0.1046511627906977 0.16666666666666669 0.276595744680851 0.21739130434782608 0.14285714285714285 0.13559322033898305 0.15555555555555556 0.20689655172413796 0.23076923076923078 0.15151515151515155 0.12280701754385966 0.23255813953488372 0.10526315789473682 0.3409090909090909 0.06557377049180328 0.23404255319148937 0.12280701754385966 0.1311475409836066 0.1126760563380282 0.08163265306122448 0.09677419354838708 0.18666666666666668 0.35294117647058826 0.12500000000000003 0.19642857142857142 0.12280701754385966 0.23529411764705885 0.2037037037037037 0.16923076923076924 0.13333333333333333
Rankings of X
25.0 19.5 50.0 44.0 2.0 30.0 31.

In [ ]:
def correlationCoefficient(X, Y):
	n = len(X)
	sum_X = 0
	sum_Y = 0
	sum_XY = 0
	squareSum_X = 0
	squareSum_Y = 0

	for i in range(n):

		# sum of elements of array X.
		sum_X = sum_X + X[i]

		# sum of elements of array Y.
		sum_Y = sum_Y + Y[i]

		# sum of X[i] * Y[i].
		sum_XY = sum_XY + X[i] * Y[i]

		# sum of square of array elements.
		squareSum_X = squareSum_X + X[i] * X[i]
		squareSum_Y = squareSum_Y + Y[i] * Y[i]

	# use formula for calculating
	# correlation coefficient.
	corr = (n * sum_XY - sum_X * sum_Y) / ((n * squareSum_X -
											sum_X * sum_X) * (n * squareSum_Y - sum_Y * sum_Y)) ** 0.5

	return corr


# Driver function
X = b2
Y =humval

# Get ranks of vector X
rank_x = rankify(X)

# Get ranks of vector y
rank_y = rankify(Y)

print("Vector X")
printVector(X)

# Print rank vector of X
print("Rankings of X")
printVector(rank_x)

# Print Vector Y
print("Vector Y")
printVector(Y)

# Print rank vector of Y
print("Rankings of Y")
printVector(rank_y)

# Print Spearmans coefficient
print("Spearman's Rank correlation: ")
print(correlationCoefficient(rank_x, rank_y))

Vector X
0.005958971446039433 0.005012066118352053 0.0134119826036175 0.008508091992197986 0.007635176756831004 0.006522480109211932 0.0070302369722512435 0.008651505658568908 0.007635176756831004 0.006370746393016614 0.008508091992197986 0.007123584229028806 0.008651505658568908 0.009458665266958812 0.012026056535665977 0.0070302369722512435 0.006601401719618529 0.006682475365147634 0.005601856221111566 0.006601401719618529 0.005240408583827657 0.006445622680040651 0.008369886090674036 0.008508091992197986 0.008107958954397138 0.0070302369722512435 0.008651505658568908 0.007123584229028806 0.006522480109211932 0.006445622680040651 0.007219734406847412 0.00895523837069896 0.004844549884681426 0.008800441009112238 0.006851456440288851 0.008369886090674036 0.007219734406847412 0.006851456440288851 0.006089542681487594 0.008107958954397138 0.006765794073844415 0.005834443396670688 0.017707583400351348 0.008236597895548708 0.007318821255882091 0.007219734406847412 0.01068594098368534 0.007

In [ ]:
def correlationCoefficient(X, Y):
	n = len(X)
	sum_X = 0
	sum_Y = 0
	sum_XY = 0
	squareSum_X = 0
	squareSum_Y = 0

	for i in range(n):

		# sum of elements of array X.
		sum_X = sum_X + X[i]

		# sum of elements of array Y.
		sum_Y = sum_Y + Y[i]

		# sum of X[i] * Y[i].
		sum_XY = sum_XY + X[i] * Y[i]

		# sum of square of array elements.
		squareSum_X = squareSum_X + X[i] * X[i]
		squareSum_Y = squareSum_Y + Y[i] * Y[i]

	# use formula for calculating
	# correlation coefficient.
	corr = (n * sum_XY - sum_X * sum_Y) / ((n * squareSum_X -
											sum_X * sum_X) * (n * squareSum_Y - sum_Y * sum_Y)) ** 0.5

	return corr


# Driver function
X = b3
Y =humval

# Get ranks of vector X
rank_x = rankify(X)

# Get ranks of vector y
rank_y = rankify(Y)

print("Vector X")
printVector(X)

# Print rank vector of X
print("Rankings of X")
printVector(rank_x)

# Print Vector Y
print("Vector Y")
printVector(Y)

# Print rank vector of Y
print("Rankings of Y")
printVector(rank_y)

# Print Spearmans coefficient
print("Spearman's Rank correlation: ")
print(correlationCoefficient(rank_x, rank_y))

Vector X
0.003021450310667881 0.002534190733998231 0.00699755614101783 0.004350728859646699 0.003892443052502081 0.0033130057697584405 0.003576787231496245 0.004426351732291069 0.003892443052502081 0.0032343789379930495 0.004350728859646699 0.00362539554513073 0.004426351732291069 0.004853788755413074 0.006235733018493471 0.003576787231496245 0.003353937970451348 0.0033960120708127314 0.002837303800303262 0.003353937970451348 0.0026513972001508965 0.003273167767208146 0.004277941779677842 0.004350728859646699 0.004140234359692155 0.003576787231496245 0.004426351732291069 0.00362539554513073 0.0033130057697584405 0.003273167767208146 0.0036755011525768646 0.004586829409382392 0.0024483209094626575 0.0045049876594265015 0.003483791410316366 0.004277941779677842 0.0036755011525768646 0.003483791410316366 0.0030888984616241413 0.004140234359692155 0.003439278654204242 0.0029571836394084324 0.009444044480187383 0.0042078271857694475 0.0037271774914214356 0.0036755011525768646 0.005509938319

In [ ]:
def correlationCoefficient(X, Y):
	n = len(X)
	sum_X = 0
	sum_Y = 0
	sum_XY = 0
	squareSum_X = 0
	squareSum_Y = 0

	for i in range(n):

		# sum of elements of array X.
		sum_X = sum_X + X[i]

		# sum of elements of array Y.
		sum_Y = sum_Y + Y[i]

		# sum of X[i] * Y[i].
		sum_XY = sum_XY + X[i] * Y[i]

		# sum of square of array elements.
		squareSum_X = squareSum_X + X[i] * X[i]
		squareSum_Y = squareSum_Y + Y[i] * Y[i]

	# use formula for calculating
	# correlation coefficient.
	corr = (n * sum_XY - sum_X * sum_Y) / ((n * squareSum_X -
											sum_X * sum_X) * (n * squareSum_Y - sum_Y * sum_Y)) ** 0.5

	return corr


# Driver function
X = b4
Y =humval

# Get ranks of vector X
rank_x = rankify(X)

# Get ranks of vector y
rank_y = rankify(Y)

print("Vector X")
printVector(X)

# Print rank vector of X
print("Rankings of X")
printVector(rank_x)

# Print Vector Y
print("Vector Y")
printVector(Y)

# Print rank vector of Y
print("Rankings of Y")
printVector(rank_y)

# Print Spearmans coefficient
print("Spearman's Rank correlation: ")
print(correlationCoefficient(rank_x, rank_y))

Vector X
0.001532306943267282 0.001281494177987741 0.0036578134373502283 0.0022259543002843573 0.001985145956776061 0.001683220673345015 0.00182032921602934 0.0022658705296251893 0.001985145956776061 0.0016424580544495955 0.0022259543002843573 0.0018456559138847355 0.0022658705296251893 0.0024924861176445524 0.0032377844519100724 0.00182032921602934 0.001704460280065439 0.0017263061359964725 0.0014373183725220461 0.001704460280065439 0.0013416708723655141 0.0016625614055660418 0.002187583864607988 0.0022259543002843573 0.002115119727234035 0.00182032921602934 0.0022658705296251893 0.0018456559138847355 0.001683220673345015 0.0016625614055660418 0.0018717829943678478 0.0023507500723084773 0.0012374665466305824 0.002307432703608697 0.001771928389729875 0.002187583864607988 0.0018717829943678478 0.001771928389729875 0.0015671617195004836 0.002115119727234035 0.0017487857563750389 0.0014991278172001078 0.005059309542957528 0.002150667228282162 0.0018987507975165793 0.0018717829943678478 0.

In [ ]:
def correlationCoefficient(X, Y):
	n = len(X)
	sum_X = 0
	sum_Y = 0
	sum_XY = 0
	squareSum_X = 0
	squareSum_Y = 0

	for i in range(n):

		# sum of elements of array X.
		sum_X = sum_X + X[i]

		# sum of elements of array Y.
		sum_Y = sum_Y + Y[i]

		# sum of X[i] * Y[i].
		sum_XY = sum_XY + X[i] * Y[i]

		# sum of square of array elements.
		squareSum_X = squareSum_X + X[i] * X[i]
		squareSum_Y = squareSum_Y + Y[i] * Y[i]

	# use formula for calculating
	# correlation coefficient.
	corr = (n * sum_XY - sum_X * sum_Y) / ((n * squareSum_X -
											sum_X * sum_X) * (n * squareSum_Y - sum_Y * sum_Y)) ** 0.5

	return corr


# Driver function
X = chRf
Y =humval

# Get ranks of vector X
rank_x = rankify(X)

# Get ranks of vector y
rank_y = rankify(Y)

print("Vector X")
printVector(X)

# Print rank vector of X
print("Rankings of X")
printVector(rank_x)

# Print Vector Y
print("Vector Y")
printVector(Y)

# Print rank vector of Y
print("Rankings of Y")
printVector(rank_y)

# Print Spearmans coefficient
print("Spearman's Rank correlation: ")
print(correlationCoefficient(rank_x, rank_y))

Vector X
0.09824218377600359 0.11974866411343983 0.03933931000081217 0.055876214814486215 0.08078216208530709 0.07571371550298614 0.07533911596503647 0.04854520576828788 0.0638758309359518 0.10841861393944745 0.0930678350892926 0.04459533580847811 0.10461504939235766 0.10680421812296732 0.025437674508483585 0.11369941656062253 0.10879017071719754 0.10271950235186471 0.07353012738186944 0.24964818596121255 0.08686237157704992 0.08064948871114716 0.07956775401486797 0.06512650226355914 0.15407177841524658 0.10256914281771495 0.0654367878459035 0.09791611737448368 0.10211359202134003 0.16386148380678714 0.08838978487775079 0.08837563499287336 0.10917812142038953 0.0843755672516079 0.05332027571835544 0.05696138764784902 0.07856587652243034 0.09992462818060045 0.10250721788635098 0.042037399214156196 0.048843113460862266 0.1509380668851631 0.05531647964685745 0.042186097462083766 0.13093746597464073 0.09415510713892873 0.04017327246643572 0.09770462368261229 0.09861227671780332 0.141534990